In [1]:
import pandas as pd
from database import Database
from datetime import datetime
from Filter.Market import Market
import os

In [15]:
db = Database()
q_data=db.get_finance_report_q()
m_data=db.get_finance_report_m()
s_data = db.get_stock('2023/09/01','2023/09/16')
mk_data = db.get_market_index('2023/09/01','2023/09/16')
company_data =db.get_company_basic()
ch_data = db.get_chips('2023/09/01','2023/09/16')

c:\Users\User\Desktop\中央資管\論文\自己的論文\Data_Browsing\database.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_from_db[columns_to_fill] = filtered_df_from_db[columns_to_fill].fillna(0)


In [16]:
# s_data
s_data

,日期,證券代碼,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),5日均價(元),10日均價(元),20日均價(元),60日均價(元),5日均量,10日均量,20日均量,60日均量
280534,2023-09-15,1101,35.30,35.35,35.05,35.10,19736.0,34.94,34.78,35.02,36.46,20375.97,24229.09,21426.40,19031.30
280535,2023-09-14,1101,35.40,35.50,35.30,35.50,18247.0,34.74,34.79,35.02,36.52,20796.22,24198.23,21977.20,18997.00
280536,2023-09-13,1101,34.85,35.30,34.85,35.30,28278.0,34.51,34.74,35.03,36.57,21542.86,26225.37,23161.27,18907.66
280537,2023-09-12,1101,34.30,34.65,34.30,34.60,17792.0,34.33,34.74,35.07,36.63,23868.47,25523.93,23248.47,19088.91
280538,2023-09-11,1101,34.10,34.20,34.05,34.20,17824.0,34.41,34.82,35.17,36.69,27601.30,26021.44,23204.80,18998.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316740,2023-09-07,9962,17.20,17.30,17.15,17.30,113.0,17.31,17.19,17.15,18.46,220.14,163.28,218.12,408.19
316741,2023-09-06,9962,17.45,17.45,17.10,17.15,177.0,17.28,17.16,17.16,18.51,219.12,165.60,252.63,417.26
316742,2023-09-05,9962,17.45,17.50,17.35,17.50,122.0,17.27,17.13,17.19,18.56,196.87,160.17,276.17,423.49
316743,2023-09-04,9962,17.25,17.55,17.15,17.35,359.0,17.18,17.06,17.24,18.60,186.42,192.18,278.87,435.86


In [17]:
columns_to_add_prefix = ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '5日均價(元)', '10日均價(元)', '20日均價(元)', '60日均價(元)', '5日均量', '10日均量', '20日均量', '60日均量']
s_data.rename(columns={col: '分析起日-' + col for col in columns_to_add_prefix}, inplace=True)
s_data

,日期,證券代碼,分析起日-開盤價(元),分析起日-最高價(元),分析起日-最低價(元),分析起日-收盤價(元),分析起日-成交量(千股),分析起日-5日均價(元),分析起日-10日均價(元),分析起日-20日均價(元),分析起日-60日均價(元),分析起日-5日均量,分析起日-10日均量,分析起日-20日均量,分析起日-60日均量
280534,2023-09-15,1101,35.30,35.35,35.05,35.10,19736.0,34.94,34.78,35.02,36.46,20375.97,24229.09,21426.40,19031.30
280535,2023-09-14,1101,35.40,35.50,35.30,35.50,18247.0,34.74,34.79,35.02,36.52,20796.22,24198.23,21977.20,18997.00
280536,2023-09-13,1101,34.85,35.30,34.85,35.30,28278.0,34.51,34.74,35.03,36.57,21542.86,26225.37,23161.27,18907.66
280537,2023-09-12,1101,34.30,34.65,34.30,34.60,17792.0,34.33,34.74,35.07,36.63,23868.47,25523.93,23248.47,19088.91
280538,2023-09-11,1101,34.10,34.20,34.05,34.20,17824.0,34.41,34.82,35.17,36.69,27601.30,26021.44,23204.80,18998.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316740,2023-09-07,9962,17.20,17.30,17.15,17.30,113.0,17.31,17.19,17.15,18.46,220.14,163.28,218.12,408.19
316741,2023-09-06,9962,17.45,17.45,17.10,17.15,177.0,17.28,17.16,17.16,18.51,219.12,165.60,252.63,417.26
316742,2023-09-05,9962,17.45,17.50,17.35,17.50,122.0,17.27,17.13,17.19,18.56,196.87,160.17,276.17,423.49
316743,2023-09-04,9962,17.25,17.55,17.15,17.35,359.0,17.18,17.06,17.24,18.60,186.42,192.18,278.87,435.86


In [18]:
ch_data

,日期,證券代碼,外資買賣超(張),投信買賣超(張),自營買賣超(張),合計買賣超(張),外資買賣超日數,投信買賣超日數,自營買賣超日數,法人買賣超日數,...,融資餘額(張),融券餘額(張),券資比,融券增減比率,融資增減比率,200-400 張(比率),400-600 張(比率),600-800 張(比率),800-1000張(比率),1000張以上(比率)
156,2023-09-01,1101,-5509.0,17.0,-286.0,-5778.0,-6,6,-1,-6,...,15309.0,3373.0,22.03,0.45,0.49,2.81,1.60,1.01,1.03,56.84
157,2023-09-04,1101,-3864.0,62.0,-70.0,-3873.0,-7,7,-2,-7,...,15385.0,3322.0,21.59,-1.51,0.50,2.81,1.60,1.01,1.03,56.84
158,2023-09-05,1101,-8772.0,30.0,123.0,-8618.0,-8,8,1,-8,...,15713.0,3883.0,24.71,16.89,2.13,2.81,1.60,1.01,1.03,56.84
159,2023-09-06,1101,-18901.0,58.0,-1760.0,-20603.0,-9,9,-1,-9,...,15503.0,3966.0,25.58,2.14,-1.34,2.81,1.60,1.01,1.03,56.84
160,2023-09-07,1101,-3825.0,56.0,-275.0,-4043.0,-10,10,-2,-10,...,15492.0,3946.0,25.47,-0.50,-0.07,2.81,1.60,1.01,1.03,56.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432633,2023-09-11,912000,-4.0,0.0,0.0,-4.0,-4,0,0,-4,...,5308.0,0.0,0.00,0.00,0.00,3.84,2.72,2.92,0.00,36.75
432634,2023-09-12,912000,-7.0,0.0,0.0,-7.0,-5,0,0,-5,...,5241.0,0.0,0.00,0.00,-1.26,3.84,2.72,2.92,0.00,36.75
432635,2023-09-13,912000,-9.0,0.0,0.0,-9.0,-6,0,0,-6,...,5076.0,0.0,0.00,0.00,-3.15,3.84,2.72,2.92,0.00,36.75
432636,2023-09-14,912000,-21.0,0.0,0.0,-21.0,-7,0,0,-7,...,4680.0,0.0,0.00,0.00,-7.80,3.84,2.72,2.92,0.00,36.75


In [19]:
columns_to_add_prefix = ['外資買賣超(張)', '投信買賣超(張)', '自營買賣超(張)', '合計買賣超(張)', '外資買賣超日數', '投信買賣超日數', '自營買賣超日數', '法人買賣超日數', '董監持股％', '董監持股數', '融資餘額(張)', '融券餘額(張)', '券資比', '融券增減比率', '融資增減比率', '200-400 張(比率)', '400-600 張(比率)', '600-800 張(比率)', '800-1000張(比率)', '1000張以上 (比率)']
ch_data.rename(columns={col: '分析起日-' + col for col in columns_to_add_prefix}, inplace=True)
ch_data

,日期,證券代碼,分析起日-外資買賣超(張),分析起日-投信買賣超(張),分析起日-自營買賣超(張),分析起日-合計買賣超(張),分析起日-外資買賣超日數,分析起日-投信買賣超日數,分析起日-自營買賣超日數,分析起日-法人買賣超日數,...,分析起日-融資餘額(張),分析起日-融券餘額(張),分析起日-券資比,分析起日-融券增減比率,分析起日-融資增減比率,分析起日-200-400 張(比率),分析起日-400-600 張(比率),分析起日-600-800 張(比率),分析起日-800-1000張(比率),1000張以上(比率)
156,2023-09-01,1101,-5509.0,17.0,-286.0,-5778.0,-6,6,-1,-6,...,15309.0,3373.0,22.03,0.45,0.49,2.81,1.60,1.01,1.03,56.84
157,2023-09-04,1101,-3864.0,62.0,-70.0,-3873.0,-7,7,-2,-7,...,15385.0,3322.0,21.59,-1.51,0.50,2.81,1.60,1.01,1.03,56.84
158,2023-09-05,1101,-8772.0,30.0,123.0,-8618.0,-8,8,1,-8,...,15713.0,3883.0,24.71,16.89,2.13,2.81,1.60,1.01,1.03,56.84
159,2023-09-06,1101,-18901.0,58.0,-1760.0,-20603.0,-9,9,-1,-9,...,15503.0,3966.0,25.58,2.14,-1.34,2.81,1.60,1.01,1.03,56.84
160,2023-09-07,1101,-3825.0,56.0,-275.0,-4043.0,-10,10,-2,-10,...,15492.0,3946.0,25.47,-0.50,-0.07,2.81,1.60,1.01,1.03,56.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432633,2023-09-11,912000,-4.0,0.0,0.0,-4.0,-4,0,0,-4,...,5308.0,0.0,0.00,0.00,0.00,3.84,2.72,2.92,0.00,36.75
432634,2023-09-12,912000,-7.0,0.0,0.0,-7.0,-5,0,0,-5,...,5241.0,0.0,0.00,0.00,-1.26,3.84,2.72,2.92,0.00,36.75
432635,2023-09-13,912000,-9.0,0.0,0.0,-9.0,-6,0,0,-6,...,5076.0,0.0,0.00,0.00,-3.15,3.84,2.72,2.92,0.00,36.75
432636,2023-09-14,912000,-21.0,0.0,0.0,-21.0,-7,0,0,-7,...,4680.0,0.0,0.00,0.00,-7.80,3.84,2.72,2.92,0.00,36.75


In [23]:
def getFliterStock(self):

        s_data =self.s_data

        #找出符合條件的公司股價資料
        stock_fliter_data = s_data[(s_data['證券代碼'].isin(self.selected_companies))]

        # 先將 DataFrame 根據 "證券代碼" 和 "日期" 進行排序
        stock_fliter_data = stock_fliter_data.sort_values(by=["證券代碼", "日期"], ascending=[True, True])

        #取得有資料的首日
        first_date = stock_fliter_data.head(1)['日期']

        #取得區間內最高價
        max_prices= stock_fliter_data.groupby('證券代碼')['最高價(元)'].max()
        max_prices = max_prices.rename('區間最高價(元)').reset_index()

        start_prices = stock_fliter_data[stock_fliter_data['日期'] == first_date.iloc[0]]
        result_df = pd.merge(max_prices, start_prices, on='證券代碼', how='left')
        # 計算 "區間最高股價變化"
        result_df['區間最大價差'] = (result_df['區間最高價(元)'] - result_df['最低價(元)'])

        # 計算 "區間最高股價變化率"
        result_df['區間最大股價變化率'] = ((result_df['區間最高價(元)'] - result_df['最低價(元)'])/result_df['最低價(元)']*100).round(2)
        return result_df  
  
def getNewStockColName(self,stock_data):
    columns_to_add_prefix = ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '5日均價(元)', '10日均價(元)', '20日均價(元)', '60日均價(元)', '5日均量', '10日均量', '20日均量', '60日均量']
    stock_data.rename(columns={col: '分析起日-' + col for col in columns_to_add_prefix}, inplace=True)
    return stock_data

def getNewChipsColName(self,ch_data):
    columns_to_add_prefix = ['外資買賣超(張)', '投信買賣超(張)', '自營買賣超(張)', '合計買賣超(張)', '外資買賣超日數', '投信買賣超日數', '自營買賣超日數', '法人買賣超日數', '董監持股％', '董監持股數', '融資餘額(張)', '融券餘額(張)', '券資比', '融券增減比率', '融資增減比率', '200-400 張(比率)', '400-600 張(比率)', '600-800 張(比率)', '800-1000張(比率)', '1000張以上 (比率)']
    ch_data.rename(columns={col: '分析起日-' + col for col in columns_to_add_prefix}, inplace=True)
    return ch_data

def getNewMarketColName(self,mk_data):
    columns_to_add_prefix = ['指數開盤價(元)', '指數最高價(元)', '指數最低價(元)', '指數收盤價(元)', '指數成交量(千股)', '指數5日均價(元)', '指數10日均價(元)', '指數20日均價(元)', '指數60日均價(元)', '指數5日均量', '指數10日均量', '指數20日均量', '指數60日均量']
    mk_data.rename(columns={col: '分析起日-' + col for col in columns_to_add_prefix}, inplace=True)
    return mk_data

In [ ]:
selected_companies=[1101,4971,9940]
start_date = pd.to_datetime('2023/9/1')
end_date = pd.to_datetime('2023/9/15')

In [ ]:
#找出符合條件的公司股價資料
stock_fliter_data = s_data[(s_data['證券代碼'].isin(selected_companies)) & (s_data['日期'] >= start_date) & (s_data['日期'] <= end_date)]
# stock_fliter_data = [(s_data['證券代碼'].isin(selected_companies)) & (s_data['日期'] >= start_date) & (s_data['日期'] <= end_date)]
# stock_fliter_data = s_data[(s_data['證券代碼'].isin(selected_companies)) ]
stock_fliter_data = stock_fliter_data.sort_values(by=["證券代碼", "日期"], ascending=[True, False])
stock_fliter_data

In [ ]:
first_date = stock_fliter_data.head(1)['日期']
first_date

In [ ]:
# max_prices_idx = stock_fliter_data.groupby('證券代碼')['最高價(元)'].idxmax()
# max_prices_with_dates = stock_fliter_data.loc[max_prices_idx]
# max_prices_with_dates
max_prices= stock_fliter_data.groupby('證券代碼')['最高價(元)'].max()
max_prices = max_prices.rename('區間最高價(元)').reset_index()
max_prices

In [ ]:
start_prices = stock_fliter_data[stock_fliter_data['日期'] == start_date]
result_df = pd.merge(max_prices, start_prices, on='證券代碼', how='left')
 # 計算 "區間最高股價變化"
result_df['區間最大價差'] = (result_df['區間最高價(元)'] - result_df['最低價(元)'])

# 計算 "區間最高股價變化率"
result_df['區間最大價差%'] = ((result_df['區間最高價(元)'] - result_df['最低價(元)'])/result_df['最低價(元)']*100).round(2)
result_df

In [ ]:
# stock_data = pd.merge(max_prices_with_dates, result_df[['證券代碼','區間最大價差','區間最大價差%']], on='證券代碼', how='left')
# stock_data.columns = ['最高價當日-' + col if col in ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '5日均價(元)', '10日均價(元)', '20日均價(元)', '60日均價(元)', '5日均量', '10日均量', '20日均量', '60日均量'] else col for col in stock_data.columns]
# stock_data

In [ ]:
# filter_stock_data=pd.merge(stock_data,company_data,on=['證券代碼'], how='left')
# filter_stock_data

In [ ]:
# max_price_mdata= pd.merge(filter_stock_data,mk_data,how='inner', left_on=['日期', '上市別'], right_on=['日期', '指數名稱'])
# max_price_mdata

In [ ]:
# max_price_mdata.columns = ['最高價當日-' + col if col in ['指數開盤價(元)', '指數最高價(元)', '指數最低價(元)', '指數收盤價(元)', '指數成交量(千股)', '指數5日均價(元)', '指數10日均價(元)', '指數20日均價(元)', '指數60日均價(元)', '指數5日均量', '指數10日均量', '指數20日均量', '指數60日均量'] else col for col in max_price_mdata.columns]
# max_price_mdata

In [ ]:
#找出時間時間條件內的第一日的大盤狀況
# first_date_mdata=mk_data[mk_data['日期']==first_date.iloc[0]]
# first_date_mdata

In [ ]:
# computed_mdata = pd.merge(
#     max_price_mdata[['證券代碼', '最高價當日-指數收盤價(元)', '指數名稱']],
#     first_date_mdata[['指數收盤價(元)', '指數名稱']],
#     on='指數名稱',
#     how='left'
# )

# computed_mdata

In [ ]:
# computed_mdata['最高價當日指數變化']=(computed_mdata['最高價當日-指數收盤價(元)'] - computed_mdata['指數收盤價(元)']).round(2)
# computed_mdata['最高價當日指數變化率']=((computed_mdata['最高價當日-指數收盤價(元)'] - computed_mdata['指數收盤價(元)'])/computed_mdata['指數收盤價(元)']*100).round(2)
# computed_mdata

In [ ]:
# flited_data_sm=pd.merge(max_price_mdata,computed_mdata[['證券代碼','最高價當日指數變化','最高價當日指數變化率']],on='證券代碼',how='left')
# flited_data_sm

In [ ]:
# max_price_chdata= pd.merge(flited_data_sm,ch_data, on=['日期', '證券代碼'],how='inner')
# max_price_chdata

In [ ]:
# max_price_chdata.columns = ['最高價當日-' + col if col in ['外資買賣超(張)',
# '投信買賣超(張)',
# '自營買賣超(張)',
# '合計買賣超(張)',
# '外資買賣超日數',
# '投信買賣超日數',
# '自營買賣超日數',
# '法人買賣超日數',	
# '董監持股％',
# '董監持股數',
# '融資餘額(張)',	
# '融券餘額(張)',	
# '券資比',
# '融券增減比率',	
# '融資增減比率',	
# '200-400 張(比率)',	
# '400-600 張(比率)',	
# '600-800 張(比率)',	
# '800-1000張(比率)',	
# '1000張以上(比率)'] else col for col in max_price_chdata.columns]
# max_price_chdata

In [ ]:
# #找出時間時間條件內的第一日的籌碼狀況
# first_date_chdata=ch_data[ch_data['日期']==first_date.iloc[0]]
# first_date_chdata

In [ ]:
# computed_chdata = pd.merge(
#     max_price_chdata[['證券代碼','最高價當日-董監持股數','最高價當日-融資餘額(張)','最高價當日-融券餘額(張)','最高價當日-200-400 張(比率)','最高價當日-400-600 張(比率)','最高價當日-600-800 張(比率)','最高價當日-800-1000張(比率)','最高價當日-1000張以上(比率)']],
#     first_date_chdata[['證券代碼', '董監持股數','融資餘額(張)','融券餘額(張)','200-400 張(比率)','400-600 張(比率)','600-800 張(比率)','800-1000張(比率)','1000張以上(比率)']],
#     on='證券代碼',
#     how='left'
# )

# computed_chdata

In [ ]:
# import numpy as np
# computed_chdata['最高價當日董監持股變化']=(computed_chdata['最高價當日-董監持股數'] - computed_chdata['董監持股數']).round(2)
# computed_chdata['最高價當日融資餘額(張)變化']=(computed_chdata['最高價當日-融資餘額(張)'] - computed_chdata['融資餘額(張)']).round(2)
# computed_chdata['最高價當日融券餘額(張)變化']=(computed_chdata['最高價當日-融券餘額(張)'] - computed_chdata['融券餘額(張)']).round(2)
# computed_chdata['最高價當日200-400 張(比率)變化']=(computed_chdata['最高價當日-200-400 張(比率)'] - computed_chdata['200-400 張(比率)']).round(2)
# computed_chdata['最高價當日400-600 張(比率)變化']=(computed_chdata['最高價當日-400-600 張(比率)'] - computed_chdata['400-600 張(比率)']).round(2)
# computed_chdata['最高價當日600-800 張(比率)變化']=(computed_chdata['最高價當日-600-800 張(比率)'] - computed_chdata['600-800 張(比率)']).round(2)
# computed_chdata['最高價當日800-1000張(比率)變化']=(computed_chdata['最高價當日-800-1000張(比率)'] - computed_chdata['800-1000張(比率)']).round(2)
# computed_chdata['最高價當日1000張以上(比率)變化']=(computed_chdata['最高價當日-1000張以上(比率)'] - computed_chdata['1000張以上(比率)']).round(2)


# computed_chdata['最高價當日董監持股變化率']=((computed_chdata['最高價當日-董監持股數'] - computed_chdata['董監持股數'])/computed_chdata['董監持股數']*100).round(2)
# computed_chdata['最高價當日融資餘額(張)變化率'] = np.where(
#     computed_chdata['融資餘額(張)'] == 0,  # 分母為0的情況
#     0,  # 當分母為0時，將結果設為0
#     ((computed_chdata['最高價當日-融資餘額(張)'] - computed_chdata['融資餘額(張)']) / computed_chdata['融資餘額(張)'] * 100).round(2)
# )
# computed_chdata['最高價當日融券餘額(張)變化率'] = np.where(
#     computed_chdata['融券餘額(張)'] == 0,  # 分母為0的情況
#     0,  # 當分母為0時，將結果設為0
#     ((computed_chdata['最高價當日-融券餘額(張)'] - computed_chdata['融券餘額(張)']) / computed_chdata['融券餘額(張)'] * 100).round(2)
# )
# computed_chdata

In [ ]:
# flited_data_smc=pd.merge(max_price_chdata,computed_chdata[['證券代碼','最高價當日董監持股變化','最高價當日融資餘額(張)變化','最高價當日融券餘額(張)變化','最高價當日200-400 張(比率)變化','最高價當日400-600 張(比率)變化','最高價當日600-800 張(比率)變化','最高價當日800-1000張(比率)變化','最高價當日1000張以上(比率)變化','最高價當日董監持股變化率','最高價當日融資餘額(張)變化率','最高價當日融券餘額(張)變化率']],on='證券代碼',how='left')
# flited_data_smc['分析首日']=first_date.iloc[0]
# flited_data_smc = flited_data_smc.rename(columns={'日期': '最高價日期'})
# flited_data_smc

In [ ]:
# m_data['月報所屬日期'] = pd.to_datetime(m_data['月報所屬日期'])
# m_data['營收發布日'] = pd.to_datetime(m_data['營收發布日'])
# q_data['季報所屬日期'] = pd.to_datetime(q_data['季報所屬日期'])
# q_data['季報發布日'] = pd.to_datetime(q_data['季報發布日'])
# flited_data_smc['分析首日'] = pd.to_datetime(flited_data_smc['分析首日'])
# m_data = m_data.sort_values(by='營收發布日', ascending=False)
# q_data = q_data.sort_values(by='季報發布日', ascending=False)
# def find_next_date(row, data, date_column, target_date_column):
#     company_id = int(row['證券代碼'])
#     date = row['分析首日']
    
#     # 從 data 中篩選出相同 '證券代碼' 的紀錄
#     correspond_company_data = data[data['證券代碼'] == company_id]
    
#     for _, m_row in correspond_company_data.iterrows():
#         if date > m_row[date_column]:
#             return m_row[target_date_column]
    
#     return None

# # 將 '月報所屬日期' 列填充為對應的 '營收所屬日期'
# flited_data_smc['月報所屬日期'] = flited_data_smc.apply(find_next_date, args=(m_data, '營收發布日', '月報所屬日期',), axis=1)

# # 將 '季報所屬日期' 列填充為對應的 '季報所屬日期'
# flited_data_smc['季報所屬日期'] = flited_data_smc.apply(find_next_date, args=(q_data, '季報發布日', '季報所屬日期',), axis=1)
# final_merged_df = pd.merge(flited_data_smc, m_data, on=['月報所屬日期', '證券代碼'], how='left').merge(q_data, on=['季報所屬日期', '證券代碼'], how='left')
# final_merged_df

In [ ]:
# datetime_cols = final_merged_df.select_dtypes(include=[np.datetime64]).columns
# datetime_cols

In [ ]:
# import pandas as pd
# import os
# start_d='2023/04/1'
# end_d='2023/06/30'
# DATA_DIR = 'data'
# file_names = ['market1025.csv', 'market24.csv']
# combined_df_from_db = pd.DataFrame()
# for file_name in file_names:
#     data_path = os.path.join(DATA_DIR, file_name)
#     df_from_db = pd.read_csv(data_path)
#     combined_df_from_db = pd.concat([combined_df_from_db, df_from_db], ignore_index=True)
# combined_df_from_db['date'] = pd.to_datetime(combined_df_from_db['date'])
# filtered_df_from_db = combined_df_from_db[(combined_df_from_db['date'] >= start_d) & (combined_df_from_db['date'] <= end_d)]
#     # 缺值補為 0
# # columns_to_fill = ['open', 'high', 'low', 'close', 'volume', 'ma5', 'ma10', 'ma20', 'ma60', 'mavol_5', 'mavol_10', 'mavol_20', 'mavol_60']
# # filtered_df_from_db[columns_to_fill] = filtered_df_from_db[columns_to_fill].fillna(0)
# # filtered_df_from_db = filtered_df_from_db.copy()

# filtered_df_from_db